In [1]:
from prettytable import PrettyTable

In [2]:
class Grammar:
    def __init__(self, grammar_str):
        self.grammar_str = '\n'.join(filter(None, grammar_str.splitlines()))
        self.grammar = {}
        self.start = None
        self.terminals = set()
        self.nonterminals = set()

        for production in list(filter(None, grammar_str.splitlines())):
            head, _, bodies = production.partition(' -> ')

            if not head.isupper():
                raise ValueError(
                    f'\'{head} -> {bodies}\': Head \'{head}\' is not capitalized to be treated as a nonterminal.')

            if not self.start:
                self.start = head

            self.grammar.setdefault(head, set())
            self.nonterminals.add(head)
            bodies = {tuple(body.split()) for body in ' '.join(bodies.split()).split('|')}

            for body in bodies:
                if '^' in body and body != ('^',):
                    raise ValueError(f'\'{head} -> {" ".join(body)}\': Null symbol \'^\' is not allowed here.')

                self.grammar[head].add(body)

                for symbol in body:
                    if not symbol.isupper() and symbol != '^':
                        self.terminals.add(symbol)
                    elif symbol.isupper():
                        self.nonterminals.add(symbol)

        self.symbols = self.terminals | self.nonterminals

In [3]:
grammar_file=open("grammar.txt","r")
G = Grammar(grammar_file.read())

In [4]:
def first_follow(G):
    def union(set_1, set_2):
        set_1_len = len(set_1)
        set_1 |= set_2

        return set_1_len != len(set_1)

    first = {symbol: set() for symbol in G.symbols}
    first.update((terminal, {terminal}) for terminal in G.terminals)
    follow = {symbol: set() for symbol in G.nonterminals}
    follow[G.start].add('$')

    while True:
        updated = False

        for head, bodies in G.grammar.items():
            for body in bodies:
                for symbol in body:
                    if symbol != '^':
                        updated |= union(first[head], first[symbol] - set('^'))

                        if '^' not in first[symbol]:
                            break
                    else:
                        updated |= union(first[head], set('^'))
                else:
                    updated |= union(first[head], set('^'))

                aux = follow[head]
                for symbol in reversed(body):
                    if symbol == '^':
                        continue
                    if symbol in follow:
                        updated |= union(follow[symbol], aux - set('^'))
                    if '^' in first[symbol]:
                        aux = aux | first[symbol]
                    else:
                        aux = first[symbol]

        if not updated:
            return first, follow

In [5]:
class SLRParser:
    def __init__(self, G):
        self.G_prime = Grammar(f"{G.start}' -> {G.start}\n{G.grammar_str}")
        self.max_G_prime_len = len(max(self.G_prime.grammar, key=len))
        self.G_indexed = []

        for head, bodies in self.G_prime.grammar.items():
            for body in bodies:
                self.G_indexed.append([head, body])

        self.first, self.follow = first_follow(self.G_prime)
        self.C = self.items(self.G_prime)
        self.action = list(self.G_prime.terminals) + ['$']
        self.goto = list(self.G_prime.nonterminals - {self.G_prime.start})
        self.parse_table_symbols = self.action + self.goto
        self.parse_table = self.construct_table()

    def CLOSURE(self, I):
        J = I

        while True:
            item_len = len(J)

            for head, bodies in J.copy().items():
                for body in bodies.copy():
                    if '.' in body[:-1]:
                        symbol_after_dot = body[body.index('.') + 1]

                        if symbol_after_dot in self.G_prime.nonterminals:
                            for G_body in self.G_prime.grammar[symbol_after_dot]:
                                J.setdefault(symbol_after_dot, set()).add(
                                    ('.',) if G_body == ('^',) else ('.',) + G_body)

            if item_len == len(J):
                return J

    def GOTO(self, I, X):
        goto = {}

        for head, bodies in I.items():
            for body in bodies:
                if '.' in body[:-1]:
                    dot_pos = body.index('.')

                    if body[dot_pos + 1] == X:
                        replaced_dot_body = body[:dot_pos] + (X, '.') + body[dot_pos + 2:]

                        for C_head, C_bodies in self.CLOSURE({head: {replaced_dot_body}}).items():
                            goto.setdefault(C_head, set()).update(C_bodies)

        return goto

    def items(self, G_prime):
        C = [self.CLOSURE({G_prime.start: {('.', G_prime.start[:-1])}})]

        while True:
            item_len = len(C)

            for I in C.copy():
                for X in G_prime.symbols:
                    goto = self.GOTO(I, X)

                    if goto and goto not in C:
                        C.append(goto)

            if item_len == len(C):
                print(C)
                return C

    def construct_table(self):
        parse_table = {r: {c: '' for c in self.parse_table_symbols} for r in range(len(self.C))}

        for i, I in enumerate(self.C):
            for head, bodies in I.items():
                for body in bodies:
                    if '.' in body[:-1]:  # CASE 2 a
                        symbol_after_dot = body[body.index('.') + 1]

                        if symbol_after_dot in self.G_prime.terminals:
                            s = f's{self.C.index(self.GOTO(I, symbol_after_dot))}'

                            if s not in parse_table[i][symbol_after_dot]:
                                if 'r' in parse_table[i][symbol_after_dot]:
                                    parse_table[i][symbol_after_dot] += '/'

                                parse_table[i][symbol_after_dot] += s

                    elif body[-1] == '.' and head != self.G_prime.start:  # CASE 2 b
                        for j, (G_head, G_body) in enumerate(self.G_indexed):
                            if G_head == head and (G_body == body[:-1] or G_body == ('^',) and body == ('.',)):
                                for f in self.follow[head]:
                                    if parse_table[i][f]:
                                        parse_table[i][f] += '/'

                                    parse_table[i][f] += f'r{j}'

                                break

                    else:  # CASE 2 c
                        parse_table[i]['$'] = 'acc'

            for A in self.G_prime.nonterminals:  # CASE 3
                j = self.GOTO(I, A)

                if j in self.C:
                    parse_table[i][A] = self.C.index(j)

        return parse_table
    def print_augmented_grammar(self):
      def fprint(text, variable):
            print(f'{text:>12}: {", ".join(variable)}')
      print('AUGMENTED GRAMMAR:')

      for i, (head, body) in enumerate(self.G_indexed):
            print(f'{i:>{len(str(len(self.G_indexed) - 1))}}: {head:>{self.max_G_prime_len}} -> {" ".join(body)}')
            print()
      fprint('TERMINALS', self.G_prime.terminals)
      fprint('NONTERMINALS', self.G_prime.nonterminals)
      fprint('SYMBOLS', self.G_prime.symbols)
    def print_first_follow(self):
        print('\nFIRST:')
        for head in self.G_prime.grammar:
            print(f'{head:>{self.max_G_prime_len}} = {{ {", ".join(self.first[head])} }}')

        print('\nFOLLOW:')
        for head in self.G_prime.grammar:
            print(f'{head:>{self.max_G_prime_len}} = {{ {", ".join(self.follow[head])} }}')

    def print_all_sets(self):
        set_number=0;
        for i in self.C:
          print()
          print("I"+str(set_number))
          for h,b in i.items():
            prod=""
            for k in b:
              for f in k:
                prod+=" "+f
        
            print(h+"->",prod)
          set_number+=1
    def print_parsing_table(self):
        def fprint(text, variable):
            print(f'{text:>12}: {", ".join(variable)}')

        def print_line():
            print(f'+{("-" * width + "+") * (len(list(self.G_prime.symbols) + ["$"]))}')

        def symbols_width(symbols):
            return (width + 1) * len(symbols) - 1

        width = max(len(c) for c in {'ACTION'} | self.G_prime.symbols) + 2
        for r in range(len(self.C)):
            max_len = max(len(str(c)) for c in self.parse_table[r].values())
            if width < max_len + 2:
                width = max_len + 2

        print('\nPARSING TABLE:')
        print(f'+{"-" * width}+{"-" * symbols_width(self.action)}+{"-" * symbols_width(self.goto)}+')
        print(f'|{"":{width}}|{"ACTION":^{symbols_width(self.action)}}|{"GOTO":^{symbols_width(self.goto)}}|')
        print(f'|{"STATE":^{width}}+{("-" * width + "+") * len(self.parse_table_symbols)}')
        print(f'|{"":^{width}}|', end=' ')

        for symbol in self.parse_table_symbols:
            print(f'{symbol:^{width - 1}}|', end=' ')

        print()
        print_line()

        for r in range(len(self.C)):
            print(f'|{r:^{width}}|', end=' ')

            for c in self.parse_table_symbols:
                print(f'{self.parse_table[r][c]:^{width - 1}}|', end=' ')

            print()

        print_line()
        print()

In [6]:
slr_parser = SLRParser(G)
slr_parser.print_augmented_grammar()

[{"STMT'": {('.', 'STMT')}, 'STMT': {('.', 'type', 'var', 'op', 'cp', 'begin', 'BODY', 'end')}}, {'STMT': {('type', '.', 'var', 'op', 'cp', 'begin', 'BODY', 'end')}}, {"STMT'": {('STMT', '.')}}, {'STMT': {('type', 'var', '.', 'op', 'cp', 'begin', 'BODY', 'end')}}, {'STMT': {('type', 'var', 'op', '.', 'cp', 'begin', 'BODY', 'end')}}, {'STMT': {('type', 'var', 'op', 'cp', '.', 'begin', 'BODY', 'end')}}, {'STMT': {('type', 'var', 'op', 'cp', 'begin', '.', 'BODY', 'end')}, 'BODY': {('.',), ('.', 'LOOP', 'BODY'), ('.', 'INITIALIZATIONLIST', 'BODY'), ('.', 'DECL', 'BODY')}, 'LOOP': {('.', 'for', 'op', 'EXPR', 'sc', 'var', 'relop', 'var', 'sc', 'INITIALIZATIONLIST', 'cp', 'begin', 'BODY', 'end')}, 'INITIALIZATIONLIST': {('.', 'var', 'eq', 'EXPR', 'sc', 'INITIALIZATIONLIST'), ('.', 'var', 'eq', 'EXPR', 'sc')}, 'DECL': {('.', 'type', 'sp', 'VARLIST', 'sc')}}, {'INITIALIZATIONLIST': {('var', '.', 'eq', 'EXPR', 'sc', 'INITIALIZATIONLIST'), ('var', '.', 'eq', 'EXPR', 'sc')}}, {'STMT': {('type', 'v

In [7]:
slr_parser.print_first_follow()


FIRST:
             STMT' = { type }
              STMT = { type }
              BODY = { type, ^, for, var }
              DECL = { type }
           VARLIST = { var }
              LOOP = { for }
INITIALIZATIONLIST = { var }
              EXPR = { num, op, var }

FOLLOW:
             STMT' = { $ }
              STMT = { $ }
              BODY = { end }
              DECL = { type, end, for, var }
           VARLIST = { sc }
              LOOP = { type, end, for, var }
INITIALIZATIONLIST = { cp, for, var, type, end }
              EXPR = { cp, oprt, sc }


In [8]:
slr_parser.print_all_sets()


I0
STMT'->  . STMT
STMT->  . type var op cp begin BODY end

I1
STMT->  type . var op cp begin BODY end

I2
STMT'->  STMT .

I3
STMT->  type var . op cp begin BODY end

I4
STMT->  type var op . cp begin BODY end

I5
STMT->  type var op cp . begin BODY end

I6
STMT->  type var op cp begin . BODY end
BODY->  . . LOOP BODY . INITIALIZATIONLIST BODY . DECL BODY
LOOP->  . for op EXPR sc var relop var sc INITIALIZATIONLIST cp begin BODY end
INITIALIZATIONLIST->  . var eq EXPR sc INITIALIZATIONLIST . var eq EXPR sc
DECL->  . type sp VARLIST sc

I7
INITIALIZATIONLIST->  var . eq EXPR sc INITIALIZATIONLIST var . eq EXPR sc

I8
STMT->  type var op cp begin BODY . end

I9
DECL->  type . sp VARLIST sc

I10
BODY->  . LOOP BODY . INITIALIZATIONLIST BODY . INITIALIZATIONLIST . BODY . DECL BODY

I11
BODY->  . LOOP BODY . INITIALIZATIONLIST BODY . . DECL BODY DECL . BODY

I12
LOOP->  for . op EXPR sc var relop var sc INITIALIZATIONLIST cp begin BODY end

I13
BODY->  . LOOP BODY . INITIALIZATIONLIST BOD

In [9]:
slr_parser.print_parsing_table()


PARSING TABLE:
+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------+
|                    |                                                                                                                                               ACTION                                                                                                                                                |                                                                       GOTO                                                                       |
|       STATE        +--------------------+-------------